## Install app requirements

In [ ]:
!pip install --upgrade pip
!pip install -r requirements.txt
!source .venv/bin/activate

___

### Load environment variables

In [ ]:
from commons.constants import *
products_url = MDB_PRODUCTS_COLLECTION_URL
embeddings_url = MDB_EMBEDDINGS_COLLECTION_URL
products_collection = MDB_PRODUCTS_COLLECTION
embeddings_collection = MDB_EMBEDDINGS_COLLECTION

## Connect to Spark Cluster using SparkConnect

In [ ]:
from analytics.sc_session import get_session
session = get_session()
session

In [ ]:
from analytics.sc_embeddings_w2v import create_product_embeddings_w2v
create_product_embeddings_w2v(session)